# Personal Messaging: 
# Routing!

![route](https://media.giphy.com/media/5hpXZ7ENJKiCKVhULE/giphy.gif)

### Now that you have most of the structure from your forms and templates, you have to guide your users to each of those pages!

The way this working example is structured is to have a page for Login, Chat Choice, and then Private messaging. 

** Keep in mind that this all is up to your discretion, and may be edited as you see fit **

# Basic routing with MongoDB, Mongoengine, and Flask

1. Each of these pages will include one of your already created forms
2. Both mongoengine and Flask data must be saved to their respective places
3. Each page will redirect to the next page and render the template you have also previously created

Example:


![Screenshot (54)](https://user-images.githubusercontent.com/51918954/63884074-25096f80-c99b-11e9-8b28-abf08ca80ad5.png)


![how](https://media.giphy.com/media/3o7btPCcdNniyf0ArS/giphy.gif)
Whaaa

### Explained:

In [ ]:
# Form part!

@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()

    if form.validate_on_submit():

In [ ]:
## Query to find if the user is already in the MongoDB database

user = db.users.find_one({'username': form.username.data})

In [ ]:
# If the user is not already saved, create a new profile

if user is None:
    user = {
        'username' : form.username.data,
        'last_seen' : datetime.datetime.now(),
        'messages_received' : None,
        'last_message_read_time' : datetime.datetime.now()
    }

Each of the above attributes for 'user' correspond to the 'users' Dictionary defined at the top of this page as well as in your database

In [ ]:
# Lastly, SAVE!

community = db.users.insert_one(user)

###### Flask Part!

In [ ]:
# Must create a new user for the Flask library as well

new_user = User(username=form.username.data) 
new_user.last_seen = datetime.datetime.now()
new_user.last_message_read_time = datetime.datetime.now()
new_user.save() ## and SAVE

In [ ]:
# OR 
new_user = User(username=form.username.data, 
                last_seen=datetime.datetime.now(), 
               last_message_read_time=datetime.datetime.now()).save()

Why do all this?

In [ ]:
# to be able to use this flask extension!

login_user(new_user, force = True, fresh = True)

login_user uses the user_loader from the first Setup lesson! It allows flask to remember who the current_user is and load them without looking through the databases!

In [ ]:
# Lastly, for real

# Send user to the next page (named chat in this example), and render 
# your login template with your form used above
return redirect(url_for('chat'))
    return render_template('cs_login.html', title='Sign In', form=form)

### LOGGED IN
![logged in](https://media.giphy.com/media/l4JyQbMga55ZDEZnG/giphy.gif)


#### Next, there must be a way for the user to choose a chat to engage in!
This example created another page with a form to do this as exemplified in the last lesson.

Other options to do this may be:
1. display their old chat members, and allow them to click buttons sending them to that chat
2. display their old chat members in a drop down list including (new)
3. click on profiles to chat with someone

etc


In [ ]:
# A simple example

@app.route('/chat', methods=['GET', 'POST'])
@login_required
def chat():

    form = RecipientForm() # choose who to chat with in a form

    if form.validate_on_submit():
        user = current_user
        recipient = form.recipient.data
        return redirect(url_for('send', recipient = recipient))

    return render_template('phonebook.html', form=form)

## Level up!

![level up](https://media.giphy.com/media/Wt7u1mKdpNOPoAKVkn/giphy.gif)

###### Now for the actual chat page!!!
We will begin set up and tie up the loose ends in the next lesson!

In [1]:
# SIMILAR TO THE LAST PAGE:
        # form,
        # create and insert into database
        # save
        # render template 

Example!

In [ ]:
@app.route('/messages', methods=['GET', 'POST'])
@login_required ########## works with flask_login to remember users ###
def send():
    
    # this example uses forms to gain message data
    form = MessageForm()
    
    # this example uses a form from the last page to gain recipient data
    recipient = request.args.get('recipient')    

In [ ]:
if form.validate_on_submit():
        user = current_user
        msg = {
            'author' : user.username,
            'recipient' : recipient,
            'body' : form.body.data,
            'timestamp' : datetime.datetime.now()
        } #mongoengine db dictionary
        
        ######## this information must be added to both databases!! ######
        chat = db.messages.insert_one(msg)
        community = db.users.insert_one(msg)

At this point, the message has been stored in its own database, and connected to it's author and recipient

In [ ]:
# Flask document update
    new_message = Message(author=current_user.username,
                            recipient=recipient,
                            body=form.body.data,
                            timestamp=datetime.datetime.now())
    
# why does this not get saved?
    # because Message is an embedded document and will be saved into a 
    # User document next lesson!

In [ ]:
# DIFFERENT FROM THE LAST PAGE
        # to be learned in the next lesson :)